# INTRODUCTION



Employee retention is critical for organizations, impacting company performance and the workplace culture. This report will explore the factors influencing an employee's decision to leave a company and building a model that predicts whether an employee leaves or not based on those factors. 

A particular focus has been placed on how demographic factors such as age, gender, and payment tier affect turnover rates.


**Objective:**

This study aims to investigate the following question:
### <center> Is an employee going to leave a company based on age, gender, and salary? </center>



This research will be conducted with reference to existing literature, including "Employee Retention Practices in Hospitality Industry in Nepal: Investigating the Moderating Effect of Management Hierarchy, Age Group and Gender" by Sthapit and Shrestha in 2018 which provide insights into the covariates of interest. By aligning our analysis with these findings, we aim to contribute to the broader understanding of employee turnover and inform organizational practices.

**Background information:**

This study has highlighted that young employees may be more prone to leaving their jobs compared to older employees (Ng et al., 2010). Additionally, the gender dynamics of employee turnover rates are differently distributed between male and female employees. "Women might have different turnover rates compared to men, influenced by organizational support and workplace environment (Hom et al., 2017).

The role of salary as a predictor of employee retention has also been a subject of extensive research. The relationship is complex and may vary depending on other factors such as job satisfaction and career advancement opporunities (Trevor et al., 1997)

## The Data Set

### Employee Retention data set: 
**Source:** https://www.kaggle.com/datasets/tawfikelmetwally/employee-dataset?resource=download

**Summary:**
- Number of observations: 4654
- Number of features: 9

**Features:**
- `education`: the degree which each employee obtain.
- `JoiningYear`: year at which the employee join the company.
- `City`: The location or city where each employee is based or works.
- `PaymentTier`: categorization of employees into different salary tiers.
- `Age`: the age of each employee, providing demographic insights.
- `Gender`: gender identity of employees, promoting diversity analysis.
- `EverBenched`: Indicates if an employee has ever been temporarily without assigned work.
- `ExperienceInCurrentDomain`: the number of years of experience employees have in their current field.
- `LeaveOrNot`: whether the employee left the company or not.

**Background:**

The data have been collected from company's HR department in 2018 which provide valuable insights into the workforce while maintaining employees privacy rights.

# METHODS and RESULTS

# DISCUSSION

# REFERENCES